# Import

In [ ]:
# Загрузка ОС и файлов для работы на Kaggle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Загрузка библиотек
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")
import pycountry
import requests
import json

from sklearn.model_selection import train_test_split

In [ ]:
# Фиксация random seed для воспроизводимости экспериментов
RS = 42

In [ ]:
# Фиксация версии пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Загрузка данных в переменные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
train = pd.read_csv(DATA_DIR+'/main_task.csv')
valid = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# Объединяем данные, помячая тренировочные
train['train'] = 1
valid['train'] = 0
valid['Rating'] = 0.
data = train.append(valid, sort=False).reset_index(drop=True)

# Первичный взгляд на данные

In [ ]:
train.sample(5)

## Описание полей

- Restaurant_id - ID
- City - Город 
- Cuisine Style - Кухня
- Ranking - Ранг ресторана относительно других ресторанов в этом городе
- Price Range - Цены в ресторане в 3 категориях
- Number of Reviews - Количество отзывов
- Reviews - 2 последних отзыва и даты этих отзывов
- URL_TA - Cтраница ресторана на 'www.tripadvisor.com' 
- ID_TA - ID ресторана в TripAdvisor
- Rating - Рейтинг ресторана

In [ ]:
train.info()

Бросается в глаза, признаки 'Cuisine Style' и 'Reviews' являют списками. В частности, в 'Reviews' есть даты. Для корректной работы необходимо извлечь из них полезную иформацию.

## Пропуски

In [ ]:
# Функция возвращяет визуализацию и относительную долю пропусков в данных
def na_values(df):
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(df.isnull(),
                yticklabels=False, 
                cbar=False,
                cmap='viridis')
    plt.show()
    
    for column in df.columns:
        value = df[column].isna().sum()
        if value > 0:
            print(column.upper())
            print('%% пропущенных данных в столбце: \t{}'.format(np.round(value/len(df[column])*100), 2))
            print('(%% от общего кол-ва данных: \t{})'.format(np.round(value/np.product(df.shape)*100), 2))
            print()

In [ ]:
na_values(train)

Более трети строк содержит пропуски. Перед тем как приступать к моделированию, будет необходимо их заполнить.

## ID дубликаты

Имеется три признака, которые должны быть уникальны для каждого ресторана:

    - 'Restaurant_id'
    - 'URL_TA'
    - 'ID_TA'
Имеет смысл проверить уникальность и удалить неинформативные признаки

In [ ]:
# Функция возвращает %% уникальных зачений для выбранных признаков
def unique_id_check(df, cols):
    for col in cols:
        print(col.upper())
        print('%% уникальных значений в столбце: \t{}'.format(np.round(len(df[col].unique())/df.shape[0]*100, 2)))
        print()

In [ ]:
unique_id_check(train, ['Restaurant_id','URL_TA','ID_TA'])

Значения не уникальны для всех трех признаков. Имеет смысл проверить, одинаковы ли дубликаты для 'URL_TA' и 'ID_TA'. И разобраться почему 'Restaurant_id' уникально меньше, чем для трети ресторанов.

In [ ]:
# Фунция возвращает дупликаты для наиболее уникального признака из списка
def cross_duplicates(df, cols):
    max_value = 0
    for col in cols:
        if len(train[col].unique()) >= max_value:
            max_value = len(train[col].unique())
            base = col
    cols.remove(base)
    dups = df[df[base].duplicated() == True][base].values
    
    for col in cols:
        count = 0
        for i in dups:
            if len(df[df[base] == i][col].unique()) > 1:
                print('Несколько значений {} для {} = {}'.format(col.upper(), 
                                                                 base.upper(), i))
                count += 1
        if count == 0:
            print('Значения {} уникальны для каждого {}'.format(col.upper(), base.upper()))

In [ ]:
cross_duplicates(train, ['Restaurant_id','URL_TA','ID_TA'])

Признаки 'URL_TA' и 'ID_TA'  дублируют друг друга. Что касается 'Restaurant_id', необходимо рассмотреть случаи, когда для 'ID_TA' данный признак разный и наоборот.

In [ ]:
# Пример, когда для одного 'ID_TA' несколько 'Restaurant_id'
train[train.ID_TA=='d4600226']

In [ ]:
# Разброс рейтинга для одинаковых 'ID_TA'
idd = train[train['ID_TA'].duplicated() == True]
idd = pd.concat([idd, train[train['ID_TA'].duplicated(keep='last') == True]])
(idd.groupby('ID_TA').Rating.max() - 
 idd.groupby('ID_TA').Rating.min()).value_counts(normalize = True)

In [ ]:
# Пример, когда для одного 'Restaurant_id' при нескольких 'ID_TA' и 'URL_TA'
# Для удобства выведены первые 5 примеров и ссылок
r_id_dups = train[train['Restaurant_id'].duplicated() == True]['Restaurant_id'].unique()
example = train[train['Restaurant_id'] == r_id_dups[0]].URL_TA.values
display(train[train['Restaurant_id'] == r_id_dups[0]].head(5))
example[:5]

In [ ]:
# HTML - запрос на tripadvisor.com, вывод заголовков страниц
for i in range(5):
    url = 'https://www.tripadvisor.com'+example[i]
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    print(page.find('h1').text)

In [ ]:
# Разброс рейтинга для одинаковых 'Restaurant_id'
rid = train[train['Restaurant_id'].duplicated() == True]
rid = pd.concat([rid, train[train['Restaurant_id'].duplicated(keep='last') == True]])
(rid.groupby('Restaurant_id').Rating.max() - 
 rid.groupby('Restaurant_id').Rating.min()).value_counts(normalize = True)

Общий 'Restaurant_id' бывает у ресторанов с разными названиями, из разных городов, с разной кухней. Разброс целевой переменной для уникального 'Restaurant_id' равен 0 только в 28% случаев и можен достигать 4. Разные 'Restaurant_id' для одного 'ID_TA' не несут ценности, т.к. в этом случае разброс целевой переменной равен 0.

Данный признак не несет ценности и подлежит удалению.

# Обработка признаков

## 1. Удаление

- Удаление дубликатов по 'ID_TA'
- Удаление 'Restaurant_id' и 'ID_TA'

Имеет смысл оставить 'URL_TA' для дальнейших запросов

In [ ]:
train = train.drop_duplicates('ID_TA')
train.drop(['Restaurant_id', 'ID_TA'], axis=1, inplace = True)

## 2. Обработка Cuisine Style

In [ ]:
# Пометка данных имеющих пропуски
train['cuisine_nan'] = pd.isna(data['Cuisine Style']).astype('uint8')

# Создание списка из строки
train['Cuisine Style'] = train['Cuisine Style'].str.findall(r'\w+')

## 3. Обработка Price Range

In [ ]:
# Уникальные значения
pr_u = data['Price Range'].unique()
pr_u

In [ ]:
# Создание словоря для преобразования класса цены в число
mapping_dict = {pr_u[0]: 2,
                pr_u[2]: 3,
                pr_u[3]: 1}

In [ ]:
# Пометка данных имеющих пропуски
train['p_range_nan'] = pd.isna(train['Price Range']).astype('uint8')

# Преобразование признака в числовой формат
train['Price Range'] = train['Price Range'].map(mapping_dict)

## 4. Обработка Reviews

In [ ]:
# Функция преобразования списка строковых дат в формат datetime
def str_list_to_date(d_list):
    result = []
    for date in d_list:
        result.append(pd.to_datetime(date, infer_datetime_format=True))
    return result

In [ ]:
# Извлечение списка дат из Reviews
train['review_dates'] = data.Reviews.str.findall(r'\d+/\d+/\d+')
train.review_dates = train.review_dates.apply(str_list_to_date)

# Извлечение списка слов из Reviews
train['wordbox'] = train.Reviews.str.lower()
train.wordbox = train.wordbox.str.findall(r'\w[a-z]+')

# Удаление исходного столбца
train.drop('Reviews', axis=1, inplace=True)

In [ ]:
def contemporary_rating(url_ta):
    try:
        url = 'https://www.tripadvisor.com'+url_ta
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = BeautifulSoup(response.text, 'html.parser')
        return float(page.find('div', class_='Ct2OcWS4').text[:3])
    except:
        return None

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
train['current_rating'] = train.URL_TA.apply(contemporary_rating)

In [ ]:
train.current_rating.value_counts()

In [ ]:
train.sample(5)

In [ ]:
train.loc[0].URL_TA

In [ ]:
url = 'https://www.tripadvisor.com'+train.loc[0].URL_TA
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

In [ ]:
detail_overview = page.find('div', class_='ppr_rup ppr_priv_detail_overview_cards').text
print(detail_overview)

In [ ]:
float(detail_overview[19:22])

In [ ]:
page.find('span', class_='r2Cf69qf').text

In [ ]:
float(page.find('div', class_='Ct2OcWS4').text[:3])

In [ ]:
page.find_all('a', class_ = '_15QfMZ2L')[1].text

In [ ]:
page.find('a', target = '_blank')

In [ ]:
for i in list(data.columns):
    display(len(data[i].unique()))

Так как уникальных значений ID_TA и URL_TA меньше 50,000, данные могут содержать дубликаты

In [ ]:
#Создание DF содежащего дубликаты по признаку ID_TA
duplicates=data[data.ID_TA.duplicated() == True].sort_values(by ='ID_TA')
dup_ID_TA = data.ID_TA.value_counts()[data.ID_TA.value_counts()>1].index
for i in ID_TA:
    data

In [ ]:
duplicates.City.unique()

Судя по всему, данные задвоеннны по City == Madrid|City == Warsaw

In [ ]:
difference_check = []
for i in list(duplicates.ID_TA):
    # Ниже вычисление разницы в целевой переменной для дубликатов
    dif = list(data[data.ID_TA == i].Rating)[0] - list(data[data.ID_TA == i].Rating)[1]
    difference_check.append(dif)
difference_check

In [ ]:
duplicates.Rating.value_counts()

Разница в целевой переменной между дублями варируется в том же диапазоне, что и переменная в срезе

In [ ]:
data[data.ID_TA == duplicates.ID_TA.iloc[0]]

In [ ]:
data[data.ID_TA == duplicates.ID_TA.iloc[1]]

Судя по всему, разница между некоторыми дублями объясняется тем, что они распределены между файлами kaggle_task и main_task.

Имеет смысл создать переменную, которая обозначит, что у строки есть дубль

### 1. Отметка дубликатов

In [ ]:
# Помечаем новыми пизнаками превые и последние дубликаты
data['1'] = data.ID_TA.duplicated(keep='first')
data['2'] = data.ID_TA.duplicated(keep='last')

In [ ]:
# Функция обрабатывает всмогательные столюцы и возвращает 1 для строк имеющих дубли
def duplicate(row):
    result = 0
    if row['1'] == True:
        result = 1
    elif row['2'] == True:
        result = 1
    return result

In [ ]:
# Применение функции
data['has_duplicate'] = data.apply(duplicate, axis = 1)

In [ ]:
# Удаление вспомогательных столбцов
data.drop(['1','2'], axis = 'columns', inplace = True)

# Cleaning and Prepping Data 
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## Анализ, преобразование и заполнение признаков

### Restaurant_id

### 2. Предположительные сети ресторанов

In [ ]:
# Создание серии содержащей ID и кол-во ресторанов
id_vc = data.groupby(['Restaurant_id']).Ranking.count()
id_vc

In [ ]:
# Добавление полученной информации в DF
data['id_amount'] = data.Restaurant_id.apply(lambda x: id_vc[x])

In [ ]:
sns.scatterplot(x="id_amount", y="Rating",
                sizes=(1, 8), linewidth=0,
                data=data)

Для крупных сетей ниже разброс ненулевых значений

### Price Range

In [ ]:
data['Price Range'].unique()

### 3. Преобразование и заполнение пропусков

In [ ]:
# Пометка данных имеющих пропуски
data['price_class_nan'] = pd.isna(data['Price Range']).astype('uint8')

In [ ]:
#Создание словоря для преобразования класса цены в число, где 1 - самые дешевые рестораны.
mapping_dict = {data['Price Range'].unique()[0]: 2,
                data['Price Range'].unique()[2]: 3,
                data['Price Range'].unique()[3]: 1}

In [ ]:
# Преобразование признака в числовой формат для работы с пропусками
data['price_class'] = data['Price Range'].map(mapping_dict)

In [ ]:
# Функция вычисления среднего класса по городу и квинтилю Ranking для города
def main_price(city, i):
    data_c = data[data.City == city]
    result = data_c[(data_c.Ranking.quantile(i) <= data_c.Ranking) & (data_c.Ranking < data_c.Ranking.quantile(i+0.2))].price_class.dropna().mean()
    return result

In [ ]:
# Функция вычисляющая скрию значений по городам для определенного квинтиля
def quantile(quantile):
    series=[]
    for i in data.City.unique():
        series.append(main_price(i, quantile))
    return series

In [ ]:
# Создание DF c итоговыми значениями
city_price = pd.DataFrame({'city': data.City.unique()})
quantiles = [x/10 for x in range(8, -1, -2)]
for i in quantiles:
    city_price[i]=quantile(i)

In [ ]:
city_price

В большинстве случаев, среднеарифметический класс соответствует самому популярному значению в DF (средний). Тем не менее, иммется ряд исключений.

In [ ]:
#Создание списка содержащего вложенный список [город-квинтиль],
#где преоблядают дешевые рестораны.
class1_price=[]
for i in quantiles:
    for j in city_price[city_price[i]<1.5].city:
        class1_price.append([j, i])
class1_price

In [ ]:
#Заполнение соответствующих значений
class1_indexes=[]
if len(class1_price)>0:
    #Цикл достает индексы строк соответствующих парам [город-квинтиль]
    for pair in class1_price:
        city = pair[0]
        Q = pair[1]
        df = data[(data.City == city) & (data[data.City == city].Ranking.quantile(Q) <= data.Ranking) & (data[data.City == city].Ranking < data.Ranking.quantile(Q+0.2))]
        #Цикл оставляет только пустые ячейки
        for i in df[df.price_class.isna()].index:
            class1_indexes.append(i)
    #Цикл заполния пустых значений
    for i in class1_indexes:
        data.xs(i)['price_class'] = 1   

In [ ]:
#Создание списка содержащего вложенный список [город-квинтиль], где преоблядают дорогие рестораны.
#Не сотря на то, что список пуст, он может быть актуален для других данных
class3_price=[]
for i in quantiles:
    for j in city_price[city_price[i]>=2.5].city:
        class1_price.append([j, i])
class3_price

In [ ]:
#Заполнение соответствующих значений аналогично дешевым ресторанам
class3_indexes=[]
if len(class3_price)>0:
    for pair in class3_price:
        city = pair[0]
        Q = pair[1]
        df = data[(data.City == city) & (data[data.City == city].Ranking.quantile(Q) <= data.Ranking) & (data[data.City == city].Ranking < data.Ranking.quantile(Q+0.2))]
        for i in df[df.price_class.isna()].index:
            class3_indexes.append(i)
    for i in class3_indexes:
        data.xs(i)['price_class'] = 1 

In [ ]:
#Заполнение оставшихся ячеек самым популярным значением
data.price_class.fillna(value = 2, inplace =True)

In [ ]:
data.sample(5)

In [ ]:
data['price_class'].hist()

In [ ]:
sns.catplot(x='price_class', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

### Cuisine Style

### 4. Преобразование и заполнение пропусков

In [ ]:
# Пометка данных имеющих пропуски
data['cuisine_nan'] = pd.isna(data['Cuisine Style']).astype('uint8')

In [ ]:
#Создание списка из строки
data['Cuisine Style'] = data['Cuisine Style'].str.findall(r'\w+\s*\w*\s*\w*\s*\w*\s*\w*')

In [ ]:
#Функция вычисляющая самую популярную кухню в городе.
#Значение "European" опущено, т.к. говорит о географическом происхождении не достаточно точно
def popular_cuisine(city):
    popular_values = pd.Series(data[data.City == city]['Cuisine Style'].dropna().sum()).value_counts()
    if popular_values.index[0] != 'European':
        result = popular_values.index[0]
    else:
        result = popular_values.index[1]
    return result

In [ ]:
#Создание вспомогательного DF для более быстрого заполнения пропусков
city_cuisine = pd.DataFrame({'city': data.City.unique()})
city_cuisine['cuisine'] = city_cuisine.city.apply(popular_cuisine)
city_cuisine

In [ ]:
#Функция заполнения пропусков популярным значением для города
def associated_cuisine(city):
    result = city_cuisine[city_cuisine.city == city].iloc[0][1]
    return result

In [ ]:
#Функция преобразования заполненных строк в список
def making_list(cell):
    if type(cell) == list:
        result = cell
    elif type(cell) == str:
        result = re.findall(r'\w+\s*\w*\s*\w*\s*\w*\s*\w*', cell)
    return result

In [ ]:
#Применение вышеописанных функций + создание столбца кол-ва кухонь
data['Cuisine Style'].fillna(value=data.City.apply(associated_cuisine), inplace=True)
data['Cuisine Style'] = data['Cuisine Style'].apply(making_list)
data['cuisine_amount'] = data['Cuisine Style'].apply(lambda x: len(x))

### Reviews

### 5. Преобразование и создание доп. парметров

In [ ]:
# Пропусков всего 2, можно заполнить самым популярным(пустым списом) значением
data.Reviews.fillna(data.Reviews.value_counts().index[0], inplace = True)

In [ ]:
# Функция преобразоватия строковой даты в формат datetime
def date_conversion(cell):
    resulting_list=[]
    for i in cell:
        if int(i[:i.find('/')])<=12:
            converted_time = datetime.strptime(i, '%m/%d/%Y')
            resulting_list.append(converted_time)
        else:
            if '/' in i[-4:]:
                converted_time = datetime.strptime(i, '%d/%m/%y')
                resulting_list.append(converted_time)
            else:
                converted_time = datetime.strptime(i, '%d/%m/%Y')
                resulting_list.append(converted_time)
    return resulting_list

In [ ]:
# Создание параметра review_dates содержащего список дат отзывов
data['review_dates'] = data.Reviews.str.findall(r'\d+/\d+/\d+')
data.review_dates = data.review_dates.dropna().apply(date_conversion)

In [ ]:
# Фунция вычисляет разницу между превым и последним отзывом
def review_t_dif(cell):
    if len(cell)>=2:
        dif=max(cell)-min(cell)
    else:
        dif=timedelta(days = 0)
    return dif

In [ ]:
# Параметр review_time_span отбражает разницу между превым и последним отзывом
data['review_time_span'] = data.review_dates.dropna().apply(review_t_dif)

In [ ]:
#Перевод результата в секунды
data.review_time_span = data.review_time_span.apply(lambda x: x.total_seconds())

In [ ]:
# Параметр visible_reviews отбражает кол-во видимых отзывов
data['visible_reviews'] = data.review_dates.apply(lambda x: len(x))

In [ ]:
data.sample(5)

In [ ]:
sns.scatterplot(x='review_time_span', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

In [ ]:
data['visible_reviews'].hist()

In [ ]:
sns.catplot(x='visible_reviews', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

### 6. Применение списков ключевых слов для данных

In [ ]:
# Параметр review_wordbox включает список слов используемых в Reviews
data.Reviews = data.Reviews.apply(lambda x: x.lower())
data['review_wordbox'] = data.Reviews.str.findall(r'\w[a-z]+')

In [ ]:
# Все слова, встречающиеся более 2 раз у ресторанов с целевой переменной от 4.5
wordbox_positive = pd.Series(data[(data['sample'] == 1)&(data.Rating>=4.5)].review_wordbox.sum())
p_words = pd.DataFrame(wordbox_positive.value_counts())
p_words = p_words[p_words[0]>2]

In [ ]:
# Все слова, встречающиеся более 2 раз у ресторанов с целевой переменной до 3.5
wordbox_negative = pd.Series(data[(data['sample'] == 1)&(data.Rating<=3.5)].review_wordbox.sum())
n_words = pd.DataFrame(wordbox_negative.value_counts())
n_words = n_words[n_words[0]>2]

In [ ]:
# Слова, уникальные для ресторанов с высоким рейтингом
positive_review_predictors = []
for i in p_words.index:
    if i not in list(n_words.index):
        positive_review_predictors.append(i)

In [ ]:
# Слова, уникальные для ресторанов с низким рейтингом
negative_review_predictors = []
for i in n_words.index:
    if i not in list(p_words.index):
        negative_review_predictors.append(i)

In [ ]:
# Функция расчитывающая рейтинг по ключевым словам. Нейтральное значение = 0.5
def valued_review_score(wordbox):
    positives = 1
    negatives = 1
    for i in wordbox:
        if i in positive_review_predictors:
            positives += 1
        elif i in negative_review_predictors:
            negatives += 1
    result = positives/(positives + negatives)
    return result

In [ ]:
# Параметр valued_review содержит условный рейтинг, расчитаный на осонове слов, содежащихся в Reviews
data['valued_review'] = data.review_wordbox.apply(valued_review_score)

In [ ]:
sns.scatterplot(x='valued_review', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

### Number of Reviews

### 7. Заполнение пропусков

In [ ]:
# Пометка данных имеющих пропуски
data['NoF_nan'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# Функция вычисления среднего класса по городу и квинтилю Ranking для города
def avg_reviews(city, i):
    data_c = data[data.City == city]
    result = data_c[(data_c.Ranking.quantile(i) <= data_c.Ranking) & (data_c.Ranking < data_c.Ranking.quantile(i+0.2))]['Number of Reviews'].dropna().mean()
    return result

In [ ]:
# Функция вычисляющая серию значений по городам для определенного квинтиля
def quantile(quantile):
    series=[]
    for i in data.City.unique():
        series.append(avg_reviews(i, quantile))
    return series

In [ ]:
# Создание DF c итоговыми значениями
city_rev = pd.DataFrame({'city': data.City.unique()})
quantiles = [x/10 for x in range(8, -1, -2)]
for i in quantiles:
    city_rev[i]=quantile(i)

In [ ]:
city_rev

In [ ]:
# Функция возвращает соответсвующее значение из city_rev по City и Ranking
def aprx_rev_amount(x):
    d = data[data.City == x['City']]
    quantiles = [n/10 for n in range(8, -1, -2)]
    for i in quantiles:
        if d.Ranking.quantile(i) <= x['Ranking'] <= d.Ranking.quantile(i+0.2):
            result = float(int(city_rev[city_rev.city == x['City']][i]))
    return result

In [ ]:
# Создание серии, содержащей индексы пустых значений с соответствующее значение из city_rev
# fillna() не ипользовал, т.к. рачеты занимали больше времени
reviews_for_nan = data[data['Number of Reviews'].isna() == True].apply(aprx_rev_amount, axis=1)

In [ ]:
# Заполнение пропусков в основном DF
for i in list(reviews_for_nan.index):
    data['Number of Reviews'].loc[i] = reviews_for_nan[i]

### Ranking

### 8. Нормолизация в зависимости от City

In [ ]:
# Создание словаря, содержащего пары город: длинна вектора Ranking по городу
v_lengths = {}
for i in list(data.City.unique()):
    v_lengths[i] = np.linalg.norm(data[data.City == i].Ranking)

In [ ]:
# Функция возражает произведение значения Ranking и длинны вектора Ranking по городу
def normalization(row):
    result = row.Ranking/v_lengths[row.City]
    return result

In [ ]:
# Параметр ranking_norm содержит нормализованный Ranking
data['ranking_norm'] = data.apply(normalization, axis = 1)

In [ ]:
sns.scatterplot(x='ranking_norm', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

### City

### 9. Создание доп. параметров на основе City

In [ ]:
#Создание списка уникальных значений City
city_state = pd.DataFrame({'city': data.City.unique()})

In [ ]:
# Функция возвращает код страны по названию города
def country(city):
    coordinates = geolocator.geocode(city)[1]
    location = geolocator.reverse(coordinates, exactly_one=True)
    address = location.raw['address']
    country = address.get('country_code', '')
    return country

In [ ]:
# Функция возвращает население по названию города и коду страны 
def city_population(row):
    city = row.city
    country = row.country
    tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
    cmd = tmp % (city, country)
    res = requests.get(cmd)
    dct = json.loads(res.content)
    info = dct['records'][0]['fields']
    if 'population' in info.keys():
        result = info['population']
    else:
        result = 0
    return result

In [ ]:
# Применение функций, перевод кода страны в верхгий регистр, замена на название страны по коду
city_state['country'] = city_state.city.apply(country)
city_state['population'] = city_state.apply(city_population, axis = 1)
city_state.country = city_state.country.apply(lambda x: x.upper())
city_state.country = city_state.country.apply(lambda x: pycountry.countries.get(alpha_2=x).name)

In [ ]:
# Список с GIT, содержащий пары страна, прилагательное
demonyms = pd.read_csv('/kaggle/input/country-adjective-pairs/demonyms.csv')

In [ ]:
# Функция возвращает список прилагательных соответствующих стране
# Если функция возващает пустой список, сокращенное название страны меняется на официальное
def adjectives(country):
    result = list(demonyms[demonyms.Aalborg == country].Aalborgenser)
    if len(result) == 0:
        alternative = pycountry.countries.get(name=country).official_name
        result = list(demonyms[demonyms.Aalborg == alternative].Aalborgenser)
    return result

In [ ]:
# Применение функции
city_state['adjectives'] = city_state.country.apply(adjectives)

In [ ]:
city_state

In [ ]:
# Функция заполняет параметр native_cuisine.
# ==1, если один из тегов Cuisine Style совпадает с прилагательными города
def native(row):
    result = 0
    for i in city_state[city_state.city == row.City].adjectives.sum():
        if i in row['Cuisine Style']:
            result = 1
    return result

In [ ]:
# Параметр native_cuisine отбражает евляется ли кухня национальной
data['native_cuisine'] = data.apply(native, axis = 1)

In [ ]:
#Создание словоря с парами город - население
mapping_population = {}
for i in city_state.index:
    mapping_population[city_state.city[i]]=city_state.population[i]

In [ ]:
# Добавление населения
# Создания параметра отнашения населения и ранка ресторана.
# Таким образом, наибольшим значением обладают высокоранговые рестораны в больших городах
data['city_population'] = data.City.map(mapping_population)
data['feature_1']=data['city_population']/data['Ranking']

In [ ]:
data.sample(5)

In [ ]:
data['native_cuisine'].hist(bins=2)

In [ ]:
sns.catplot(x='native_cuisine', y='Rating',
            kind="violin", bw=.1, cut=0,
            data=data)

In [ ]:
data['city_population'].hist(bins=10)

In [ ]:
sns.scatterplot(x='feature_1', y='Rating',
                sizes=(1, 8), linewidth=0,
                data=data)

# Промежуточный итог

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
correlation = data.corr()
ax = plt.subplots(figsize=(15, 15))
sns.heatmap(correlation, annot=True, cmap='vlag', linewidths=1, center=0)

## 10. Dummy-variables

![](https://i.imgur.com/mtimFxh.png)

### 10.1 City

In [ ]:
# Создаем сетку
data = pd.get_dummies(data, columns=[ 'City'], dummy_na=False)

In [ ]:
data.sample(3)

### 10.2 Cuisine Style

In [ ]:
# Функция для заполнения категории Cuisine Style
def dummy_cuisine(cell):
    if item in cell:
        return 1
    return 0

In [ ]:
# Список уникальных значений переменной
all_cuisines = pd.Series(data['Cuisine Style'].sum()).unique()

In [ ]:
# Применение функции
for item in all_cuisines:
    data[item] = data['Cuisine Style'].apply(dummy_cuisine)

In [ ]:
data.sample(3)

### 10.3 Price Range

In [ ]:
# Создаем сетку
data = pd.get_dummies(data, columns=[ 'price_class'], dummy_na=False)

In [ ]:
data.sample(3)

> # Предобработка

Не стал писать футкцию для предобработки, т.к. задействованно достаточно много сторонних источников и вспомогательных таблиц.

## Очистка от строк/списков

In [ ]:
object_columns = [s for s in data.columns if data[s].dtypes == 'object']

In [ ]:
object_columns

In [ ]:
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
df_preproc = data
df_preproc.sample(5)

In [ ]:
df_preproc.info()

## Разделение данных

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Проверка
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
adjusted_submission=[]
for i in predict_submission:
    if i%1>=0.75:
        adjusted_submission.append(int(i)+1)
    elif 0.75>i%1>=0.25:
        adjusted_submission.append(int(i)+0.5)
    else:
        adjusted_submission.append(int(i))
adjusted_submission

In [ ]:
sample_submission['Rating'] = adjusted_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)